# Comparison Of Scenarios

In [1]:
import pandas as pd
import altair as alt
import numpy as np

## Import Scenario 1

In [2]:
def read_logs1(experiment:int):
    tx_logs = pd.read_csv(f"../logs/s1_v{experiment}_transmitter.csv", sep=';')
    rx_logs = pd.read_csv(f"../logs/s1_v{experiment}_receiver.csv", sep=';')
    
    tx_logs["ROLE"] = ["Tx"]*tx_logs.shape[0]
    rx_logs["ROLE"] = ["Rx"]*rx_logs.shape[0]
    
    tx_logs["EXPERIMENT"] = [experiment]*tx_logs.shape[0]
    rx_logs["EXPERIMENT"] = [experiment]*rx_logs.shape[0]
    
    tx_logs["SCENARIO"] = [1]*tx_logs.shape[0]
    rx_logs["SCENARIO"] = [1]*rx_logs.shape[0]
    
    return pd.concat([tx_logs, rx_logs], ignore_index=True)

In [3]:
def read_experiments1():
    experiments = pd.read_csv("../logs/s1_experiments.csv", sep=';')
    
    experiments["VELOCITY"] = (2*np.pi*experiments["RADIUS_M"]/experiments["DURATION_S"])*experiments["CYCLE"]
    
    experiments["SCENARIO"] = [1]*experiments.shape[0]
    
    return experiments

In [4]:
logs1 = pd.DataFrame([], columns=['SEQ', 'ACK', 'TIMESTAMP', 'PAYLOAD', 'RSSI', 'LQI', 'ROLE', 'EXPERIMENT'])

for experiment in range(1, 7):
    logs1 = pd.concat([logs1, read_logs1(experiment)], ignore_index=True)

tx_logs1 = logs1[logs1["ROLE"]=="Tx"]
rx_logs1 = logs1[logs1["ROLE"]=="Rx"]

experiments1 = read_experiments1()

logs_exps1 = pd.merge(logs1, experiments1, on=["EXPERIMENT", "SCENARIO"])

In [18]:
RTTs1 = pd.DataFrame((tx_logs1[tx_logs1["ACK"]==240].set_index(["SEQ", "EXPERIMENT"])["TIMESTAMP"] - tx_logs1[tx_logs1["ACK"]==0].set_index(["SEQ", "EXPERIMENT"])["TIMESTAMP"]))
RTTs1.columns=["RTT"]
RTTs1 = RTTs1.reset_index()
RTTs1["SCENARIO"] = [1]*RTTs1.shape[0]

In [6]:
logs_exps1

,SEQ,ACK,TIMESTAMP,PAYLOAD,RSSI,LQI,ROLE,EXPERIMENT,SCENARIO,MOVING_ROLE,PAYLOAD_LENGTH,RADIUS_M,DURATION_S,CYCLE,VELOCITY
0,4,0,2937,QMTBLCABW,-70,108,Tx,1,1.0,Tx,10,10,39.0,1.0,1.611073
1,4,240,2944,NaN,-71,107,Tx,1,1.0,Tx,10,10,39.0,1.0,1.611073
2,5,0,3065,GMSCRNOIA,-71,107,Tx,1,1.0,Tx,10,10,39.0,1.0,1.611073
3,5,240,3071,NaN,-71,108,Tx,1,1.0,Tx,10,10,39.0,1.0,1.611073
4,6,0,3193,FTLFPCUQF,-71,108,Tx,1,1.0,Tx,10,10,39.0,1.0,1.611073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,53,240,9777,NaN,-81,107,Rx,6,1.0,Rx,10,30,52.0,0.5,1.812457
988,54,0,9905,CIXNDHETY,-82,107,Rx,6,1.0,Rx,10,30,52.0,0.5,1.812457
989,54,240,9905,NaN,-82,107,Rx,6,1.0,Rx,10,30,52.0,0.5,1.812457
990,55,0,10033,CWWNSLCHF,-76,107,Rx,6,1.0,Rx,10,30,52.0,0.5,1.812457


In [7]:
RTTs1

,SEQ,EXPERIMENT,RTT
0,4,1,7
1,5,1,6
2,6,1,9
3,7,1,4
4,8,1,10
...,...,...,...
243,51,6,8
244,52,6,7
245,53,6,9
246,54,6,5


## Import Scenario 2

In [8]:
def read_logs2(experiment:int):
    tx_logs = pd.read_csv(f"../logs/s2_v{experiment}_transmitter.csv", sep=';')
    tc_logs = pd.read_csv(f"../logs/s2_v{experiment}_transceiver.csv", sep=';')
    rx_logs = pd.read_csv(f"../logs/s2_v{experiment}_receiver.csv", sep=';')
    
    tx_logs["ROLE"] = ["Tx"]*tx_logs.shape[0]
    tc_logs["ROLE"] = ["TC"]*tc_logs.shape[0]
    rx_logs["ROLE"] = ["Rx"]*rx_logs.shape[0]
    
    tx_logs["EXPERIMENT"] = [experiment]*tx_logs.shape[0]
    tc_logs["EXPERIMENT"] = [experiment]*tc_logs.shape[0]
    rx_logs["EXPERIMENT"] = [experiment]*rx_logs.shape[0]
    
    tx_logs["SCENARIO"] = [2]*tx_logs.shape[0]
    tc_logs["SCENARIO"] = [2]*tc_logs.shape[0]
    rx_logs["SCENARIO"] = [2]*rx_logs.shape[0]
    
    return pd.concat([tx_logs, tc_logs, rx_logs], ignore_index=True)

In [9]:
def read_experiments2():
    experiments = pd.read_csv("../logs/s2_experiments.csv", sep=';')
    
    experiments["INNER_VELOCITY"] = (2*np.pi*experiments["INNER_RADIUS_M"]/experiments["DURATION_S"])*experiments["CYCLE"]
    experiments["OUTER_VELOCITY"] = (2*np.pi*experiments["OUTER_RADIUS_M"]/experiments["DURATION_S"])*experiments["CYCLE"] * (experiments["RELATIVE_MOVEMENT"] != "Tx stationary")
    experiments["STATIONARY_INNER_VELOCITY"] = experiments["INNER_VELOCITY"] * (experiments["RELATIVE_MOVEMENT"] == "Tx stationary")
    
    experiments["SCENARIO"] = [2]*experiments.shape[0]
    
    return experiments

In [10]:
logs2 = pd.DataFrame([], columns=['SEQ', 'ACK', 'TIMESTAMP', 'PAYLOAD', 'RSSI', 'LQI', 'ROLE', 'EXPERIMENT'])

for experiment in range(1, 6):
    logs2 = pd.concat([logs2, read_logs2(experiment)], ignore_index=True)

experiments2 =  read_experiments2()
logs_exps2 = pd.merge(logs2, experiments2, on="EXPERIMENT")

tx_logs2 = logs_exps2[logs_exps2["ROLE"] == "Tx"]
tc_logs2 = logs_exps2[logs_exps2["ROLE"] == "TC"]
rx_logs2 = logs_exps2[logs_exps2["ROLE"] == "Rx"]

In [11]:
logs_exps2

,SEQ,ACK,TIMESTAMP,PAYLOAD,RSSI,LQI,ROLE,EXPERIMENT,SCENARIO_x,PAYLOAD_LENGTH,INNER_RADIUS_M,OUTER_RADIUS_M,RELATIVE_MOVEMENT,DURATION_S,CYCLE,INNER_VELOCITY,OUTER_VELOCITY,STATIONARY_INNER_VELOCITY,SCENARIO_y
0,5,0,2554,GMSCRNOIA,-84,107,Tx,1,2.0,10,15,30,in line,59.8,0.5,0.788025,1.57605,0.000000,2
1,5,240,2557,NaN,-79,108,Tx,1,2.0,10,15,30,in line,59.8,0.5,0.788025,1.57605,0.000000,2
2,5,255,2563,NaN,-80,107,Tx,1,2.0,10,15,30,in line,59.8,0.5,0.788025,1.57605,0.000000,2
3,6,0,2682,FTLFPCUQF,-80,107,Tx,1,2.0,10,15,30,in line,59.8,0.5,0.788025,1.57605,0.000000,2
4,6,240,2691,NaN,-89,107,Tx,1,2.0,10,15,30,in line,59.8,0.5,0.788025,1.57605,0.000000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2684,26,0,12282,VDPJZBKQEXISGXJTDNUNDTGADIVFKLKBLMVZDQY,-70,107,Rx,5,2.0,40,5,10,Tx stationary,23.6,1.0,1.331183,0.00000,1.331183,2
2685,27,255,12409,NaN,-75,107,Rx,5,2.0,40,5,10,Tx stationary,23.6,1.0,1.331183,0.00000,1.331183,2
2686,27,0,12409,VAWWJETYXZHQQELZOSDEKSXPJGAMABUCMHRQOOB,-75,107,Rx,5,2.0,40,5,10,Tx stationary,23.6,1.0,1.331183,0.00000,1.331183,2
2687,28,255,12538,NaN,-83,108,Rx,5,2.0,40,5,10,Tx stationary,23.6,1.0,1.331183,0.00000,1.331183,2


In [16]:
RTTs2 = pd.DataFrame((tx_logs2[tx_logs2["ACK"]==255].set_index(["EXPERIMENT", "SEQ"])["TIMESTAMP"] - tx_logs2[tx_logs2["ACK"]==0].set_index(["EXPERIMENT", "SEQ"])["TIMESTAMP"]))
RTTs2.columns=["RTT"]
RTTs2 = RTTs2.reset_index()

RTTs2["FROM"] = ["Tx"]*RTTs2.shape[0]
RTTs2["TO"] = ["Rx"]*RTTs2.shape[0]
RTTs2["SCENARIO"] = [2]*RTTs2.shape[0]

## Comparison

In [95]:
from typing import Tuple, List

def select_experiments(data:pd.DataFrame, sc_exps:List[Tuple]):
    return data[data[["SCENARIO", "EXPERIMENT"]].apply(tuple, 1).isin(sc_exps)]

experiments = pd.concat([experiments1.rename(columns={"RADIUS_M":"OUTER_RADIUS_M"}), experiments2])

In [96]:
experiments

,EXPERIMENT,MOVING_ROLE,PAYLOAD_LENGTH,OUTER_RADIUS_M,DURATION_S,CYCLE,VELOCITY,SCENARIO,INNER_RADIUS_M,RELATIVE_MOVEMENT,INNER_VELOCITY,OUTER_VELOCITY,STATIONARY_INNER_VELOCITY
0,1,Tx,10,10,39.0,1.0,1.611073,1,NaN,NaN,NaN,NaN,NaN
1,2,Rx,10,10,41.8,1.0,1.503154,1,NaN,NaN,NaN,NaN,NaN
2,3,Rx,40,10,37.5,1.0,1.675516,1,NaN,NaN,NaN,NaN,NaN
3,4,Rx,40,10,15.6,1.0,4.027683,1,NaN,NaN,NaN,NaN,NaN
4,5,Rx,40,30,52.5,0.5,1.795196,1,NaN,NaN,NaN,NaN,NaN
5,6,Rx,10,30,52.0,0.5,1.812457,1,NaN,NaN,NaN,NaN,NaN
0,1,NaN,10,30,59.8,0.5,NaN,2,15.0,in line,0.788025,1.576050,0.000000
1,2,NaN,40,30,68.8,0.5,NaN,2,15.0,in line,0.684940,1.369881,0.000000
2,3,NaN,40,30,74.7,1.0,NaN,2,20.0,Tx stationary,1.682245,0.000000,1.682245
3,4,NaN,40,30,43.5,1.0,NaN,2,10.0,Tx stationary,1.444410,0.000000,1.444410


In [66]:
RTTs = pd.concat([RTTs1, RTTs2[["EXPERIMENT", "SCENARIO", "RTT"]]])

In [67]:
RTTs

,SEQ,EXPERIMENT,RTT,SCENARIO
0,4.0,1,7,1
1,5.0,1,6,1
2,6.0,1,9,1
3,7.0,1,4,1
4,8.0,1,10,1
...,...,...,...,...
269,NaN,5,16,2
270,NaN,5,12,2
271,NaN,5,11,2
272,NaN,5,12,2


In [101]:
selected = select_experiments(RTTs, [(1, 3), (2, 5), (1, 5), (2, 2)])

alt.Chart(pd.merge(selected, experiments), title="Comparison of Round Trip Times In Scenarios").transform_calculate(
    sc_exp = "'s' + datum.SCENARIO + ' e' + datum.EXPERIMENT"
).mark_bar().encode(
    x=alt.X("sc_exp:N", title="Experiment"),
    y="average(RTT):Q",
    column="OUTER_RADIUS_M"
)

alt.Chart(...)

In [107]:
selected1 = select_experiments(RTTs, [(1, 3), (2, 5)])
selected2 = select_experiments(RTTs, [(1, 5), (2, 2)])
selected3 = select_experiments(RTTs, [(1, 6), (2, 1)])

alt.hconcat(
    
    alt.Chart(pd.merge(selected1, experiments)).transform_calculate(
        sc_exp = "'s' + datum.SCENARIO + ' e' + datum.EXPERIMENT"
    ).mark_bar().encode(
        x=alt.X("sc_exp:N", title="Experiment"),
        y="average(RTT):Q"
    ),
    
    alt.Chart(pd.merge(selected2, experiments)).transform_calculate(
        sc_exp = "'s' + datum.SCENARIO + ' e' + datum.EXPERIMENT"
    ).mark_bar().encode(
        x=alt.X("sc_exp:N", title="Experiment"),
        y="average(RTT):Q"
    ),
    
    alt.Chart(pd.merge(selected3, experiments)).transform_calculate(
        sc_exp = "'s' + datum.SCENARIO + ' e' + datum.EXPERIMENT"
    ).mark_bar().encode(
        x=alt.X("sc_exp:N", title="Experiment"),
        y="average(RTT):Q"
    ),
    
    title="Comparison of Round Trip Times In Scenarios"
)

alt.HConcatChart(...)